# Тема 9: Прогнозирование временных рядов с помощью Scikit‑learn — расширенная версия
## Теория: использование ML для прогнозирования
Методы машинного обучения могут быть эффективны, когда временной ряд:
- имеет нелинейные зависимости,
- обладает сложной сезонностью,
- зависит от большого количества контекстных факторов.

В отличие от ARIMA/ETS, ML‑модели:
- не предполагают стационарности,
- хорошо работают с большим числом признаков,
- способны учитывать нелинейности,
- часто превосходят классические модели при правильной генерации фичей.

**Какие модели работают лучше всего:**
- LinearRegression — слабая базовая линия.
- RandomForest — отлично работает на средних объёмах данных.
- Gradient Boosting (XGBoost/LightGBM) — обычно лучшие результаты.
- KNN — хорошо работает на коротких рядах.

Большая часть успеха — в том, насколько хорошо подготовлены признаки.

## 1. Генерация данных и признаков

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error

np.random.seed(42)
n=200
t=np.arange(n)
trend=0.1*t
daily=5*np.sin(2*np.pi*t/24)
weekly=3*np.sin(2*np.pi*t/168)
noise=np.random.normal(0,2,n)
traffic=100+trend+daily+weekly+noise

df=pd.DataFrame({'time':pd.date_range('2023-01-01',periods=n,freq='H'),'traffic':traffic}).set_index('time')

def add_lags(data, col, lags):
    for l in lags:
        data[f'{col}_lag{l}']=data[col].shift(l)
    return data

df = add_lags(df.copy(),'traffic',list(range(1,25)))
df['hour']=df.index.hour
df['dayofweek']=df.index.dayofweek
df['rolling_mean_6']=df['traffic'].rolling(6).mean()
df.dropna(inplace=True)

df.head()

## 2. Базовое сравнение моделей

In [ ]:
X=df.drop('traffic',axis=1)
y=df['traffic']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=False)

models={
 'LinearRegression': LinearRegression(),
 'RandomForest': RandomForestRegressor(n_estimators=300),
 'Ridge': Ridge(alpha=1.0),
 'Lasso': Lasso(alpha=0.1)
}

results={}
for name, model in models.items():
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    results[name] = {
        'MAE': mean_absolute_error(y_test,pred),
        'RMSE': mean_squared_error(y_test,pred)**0.5
    }

results

## 3. Визуализация прогнозов

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test.values,label='real')
for name, model in models.items():
    plt.plot(model.predict(X_test),label=name)
plt.legend()
plt.show()

## 4. Задания
**Задание 1.** Подберите гиперпараметры RandomForest (n_estimators, max_depth).

**Задание 2.** Реализуйте скользящее прогнозирование (rolling forecast origin):
- обучаем модель на окне `[:t]`,
- предсказываем `t+1`,
- добавляем в окно.

**Задание 3.** Сравните MAE обычного train/test и rolling‑прогноза.

**Задание 4.** Добавьте модель KNNRegressor и сравните её.

**Задание 5. Мини‑проект:**
- создайте 3 набора признаков (как в уроке 8),
- обучите RandomForest,
- сравните RMSE,
- сделайте вывод, какие признаки наиболее полезны.